# TP : SYSTEMES DE RECOMMANDATION 

**Nom et Prénom:**   BAI Yunzhi

## 1 Présentation du modèle

### Question 1.1

**Remarques:** Ici, j'ai fait des copies des fonctions fournis (déjà complétés par moi-même). C'est pour les lire et utiliser plus facile. Merci de ne pas supprimer cette partie (ni import movielensutils). Sinon, ce programme ne va pas bien marcher. 

In [2]:
import movielensutils
import numpy as np
from scipy import sparse

def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0]-1, data[:, 1]-1)), dtype=bool )
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask


def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))
    
    grad_P = rho* P - (Q0.T).dot(tmp)

    return val, grad_P


def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = rho* P - (Q.T).dot(tmp)

    grad_Q = rho* Q - (tmp).dot(P.T)

    return val, grad_P, grad_Q


def total_objective_vectorized(PQvec, R, mask, rho):
    """
    Vectorisation de la fonction precedente de maniere a ne pas
    recoder la fonction gradient
    """

    # reconstruction de P et Q
    n_items = R.shape[1]
    n_users = R.shape[0]
    F = PQvec.shape[0] / (n_items + n_users)
    Pvec = PQvec[0:n_items*F]
    Qvec = PQvec[n_items*F:]
    P = np.reshape(Pvec, (F, n_items))
    Q = np.reshape(Qvec, (n_users, F))

    val, grad_P, grad_Q = total_objective(P, Q, R, mask, rho)
    return val, np.concatenate([grad_P.ravel(), grad_Q.ravel()])

In [3]:
R, mask = movielensutils.load_movielens('ml-100k/u.data')

In [4]:
print R.shape, mask.shape

(943, 1682) (943, 1682)


In [5]:
R_mini, mask_mini=movielensutils.load_movielens('ml-100k/u.data',minidata=True)
print R_mini.shape, mask_mini.shape

(100, 200) (100, 200)


**Remarques:** On a bien vérifié que les tailles de R et mask sont bien (943,1682). Et l'option "minidata" est juste pour générer les données de tailles réduits, et la taille est (100,200). 

### Question 1.2

In [6]:
import numpy.linalg as NL
mask01=np.ones([943,1682])[mask]
nombreTotaleDeNotes=NL.norm(mask01)**2
print nombreTotaleDeNotes

100000.0


**Remarques:** 
1. Il y a 943 utilisateurs.
2. Il y a 1682 films.
3. Il y a 100000 notes qui sont donnés par les utilisateurs sur les films.

### Question 1.3

**Reponses:**
1. La fonction objectif n'est pas convexe. On a un contre-exemple pour convexe: on suppose rho=0, P et Q sont de dimension 1; pour r!=1, on a f(r,1)=f(1,r)=0; et f((1+r)/2,(1+r)/2)>0. Donc, on a un contre exemple. Donc, elle n'est pas convexe.

2.  tmp = (R - Q.dot(P)) * mask

    grad_P = rho* P - (Q.T).dot(tmp)

    grad_Q = rho* Q - (tmp).dot(P.T)
    
3. Les dérivés des gradients ne sont pas bornés. Donc il n'est pas lipschitzien. Donc, il n'a pas une constante de lipschitzien.


## 2 Trouve P quand Q0 est fixé.

**Remarques:** Dans la suite, on initialise Q0 et P0 par la méthode decomposition svd. De plus, comme ce que le sujet donne, rho=0.1 et C=10

In [7]:
from scipy.sparse.linalg import svds
U, s, V = svds(R, k=10)
Q0=U[:,0:10]
P0=V[0:10,:]
print Q0.shape, P0.shape

rho=0.1

(943, 10) (10, 1682)


### Question 2.1

**Reponses:**
1. La fonction n'est pas convexe. Suppose que Q0 est assez petit devant P1 et P2(on peut supposer Q0=>0 pour simplifier), avec P1 et P2 les deux points qu'on choit, on peut trouver facilement que la fonction n'est pas convexe.

2.  tmp = (R - Q0.dot(P)) * mask

    grad_P = rho* P - (Q0.T).dot(tmp)

3. La dérivés du gradient est bornée maintenant. On peut obtenir une constante de lipschitzien = rho+NL.norm((Q0.T).dot(Q0)


### Question 2.2

In [8]:
#Q est matrix de utilisateur, P est matrix de items.
#Q(u,c) u=943; P(c,i), I=1682
from scipy.optimize import check_grad

def func(P):
    P=np.reshape(P,(10,1682))
    tmp = (R - Q0.dot(P))*mask
    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))
    return val

def grad(P):
    P=np.reshape(P,(10,1682))
    tmp = (R - Q0.dot(P))*mask
    grad_P = rho * P - (Q0.T).dot(tmp)
    print("La taille du grad_P:"+str(grad_P.shape))
    print("La norme du grad_P:"+str(NL.norm(grad_P)))
    return grad_P.ravel()


print check_grad(func, grad, P0.ravel())

La taille du grad_P:(10, 1682)
La norme du grad_P:799.115664685
1.47903090092


**Remarques:** On voit que la différence est environ 1.46. Comme grad_P est de taille (10,1682), et la valeur du grad_p est presque 800 qui est beaucoup plus grande que 1.46 , cette différence est négligéable. Donc, ici on peut dire notre exprission du gradient est correct.

### Question 2.3

**Remarques:**
Ici, j'introduit un nouveau paramètre "nombreIteration" afin d'ajouter une autre type de critère de conditions d'arrêt. C'est pour évider que l'algorithme déroule pendant trop de temps.

In [9]:
def gradient(g, P0, gamma, epsilon, nombreIteration=False):
    if nombreIteration==False :
        P=P0
        val, grad=g(P)
        nombre=0
        while(True):
            if NL.norm(grad)<=epsilon:
                return val, P, nombre
            else:
                nombre=nombre+1
                P=P-gamma*grad
                val, grad=g(P)
    else:          
        P=P0
        val, grad=g(P)
        nombre=0
        while(True):
            if NL.norm(grad)<=epsilon or nombre>=nombreIteration:
                return val, P, nombre
            else:
                nombre=nombre+1
                P=P-gamma*grad
                val, grad=g(P)
        


### Question 2.4

In [10]:
epsilon=1.
for gamma in np.logspace(-5,1,7):
    val,P_,nombre=gradient(g=lambda P:movielensutils.objective(P,Q0=Q0,R=R,mask=mask,rho=rho),P0=P0,gamma=gamma,epsilon=epsilon,nombreIteration=500)
    print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre)+"    gamma: "+str(gamma))

valeur optimale: 684334.098444    nombre d'itération: 0    gamma: 1e-05
valeur optimale: 684334.098444    nombre d'itération: 0    gamma: 0.0001
valeur optimale: 684334.098444    nombre d'itération: 0    gamma: 0.001
valeur optimale: 684334.098444    nombre d'itération: 0    gamma: 0.01
valeur optimale: 684334.098444    nombre d'itération: 0    gamma: 0.1
valeur optimale: 684334.098444    nombre d'itération: 0    gamma: 1.0
valeur optimale: 684334.098444    nombre d'itération: 0    gamma: 10.0


**Remarques:** On trouve que les meilleur gammas sont 0.01, 0.1, 1. Donc, on peut choisir un des trois valeurs. De plus, on peut voir que si le pas est trop petit, la vitesse de convergence est très lente. Si le pas est trop grand, on ne peut pas faire fonctionner notre algorithme correctement.

**Ensuite**, on utilise 1/L0 pour le pas, avec L0 le constant lipschitzien pour Q0 fixé.

In [10]:
epsilon=1.
constantLipschitzien=rho+NL.norm((Q0.T).dot(Q0))
gamma_donne=1./constantLipschitzien
print("gamma utilisé:"+str(gamma_donne))
val,P_,nombre=gradient(g=lambda P:movielensutils.objective(P,Q0=Q0,R=R,mask=mask,rho=rho),P0=P0,gamma=gamma_donne,epsilon=epsilon,nombreIteration=500)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

gamma utilisé:0.306534300317
valeur optimale: 215965.602538    nombre d'itération: 90


## 3 Raffinements algorithmiques pour le problème à Q0 fixé


### Question 3.1

**REMARQUE:** Ici, on utilise "Line Search"

In [11]:
from scipy import optimize

def objectiveGamma(gamma,X0,direction,f):
    X=X0 + gamma*direction
    val, grad_0 = f(X)
    
    return val


def gradient_LinearSearch(g, pointDepart, epsilon, nombreIteration=False):
    if nombreIteration==False :
        pointCourant=pointDepart
        val, grad=g(pointCourant)
        nombre=0
        while(True):
            if NL.norm(grad)<=epsilon:
                return val,pointCourant,nombre
            else:
                nombre=nombre+1
                direction=-grad
                gamma=optimize.brent(lambda gamma:objectiveGamma(gamma,X0=pointCourant,direction=direction,f=g))
                pointCourant=pointCourant+gamma*direction
                val, grad=g(pointCourant)
    else:          
        pointCourant=pointDepart
        val, grad=g(pointCourant)
        nombre=0
        while(True):
            if NL.norm(grad)<=epsilon or nombre>=nombreIteration:
                return val, pointCourant,nombre
            else:
                nombre=nombre+1
                direction=-grad
                gamma=optimize.brent(lambda gamma:objectiveGamma(gamma,X0=pointCourant,direction=direction,f=g))
                pointCourant=pointCourant+gamma*direction
                val, grad=g(pointCourant)

In [12]:
val,P_optimal,nombre=gradient_LinearSearch(g=lambda P:movielensutils.objective(P,Q0=Q0,R=R,mask=mask,rho=rho),pointDepart=P0,epsilon=epsilon,nombreIteration=500)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 215963.767755    nombre d'itération: 17


### Question 3.2

**Remarque:** Ici, on utilise la méthode "gradient conjugé pour une fonction quelconque".

In [13]:
def gradient_Conjuge(g, pointDepart, epsilon, nombreIteration=False):
    if nombreIteration==False :
        pointCourant=pointDepart
        val, grad=g(pointCourant)
        direction=-grad
        nombre=0
        while(True):
            if NL.norm(grad)<=epsilon:
                return val, pointCourant, nombre
            else:
                nombre=nombre+1
                gamma=optimize.brent(lambda gamma:objectiveGamma(gamma,X0=pointCourant,direction=direction,f=g))
                pointCourant=pointCourant+gamma*direction
                norm1=NL.norm(grad)**2
                val, grad=g(pointCourant)
                norm2=NL.norm(grad)**2
                b=norm2/norm1
                direction=-grad+b*direction
    else:          
        pointCourant=pointDepart
        val, grad=g(pointCourant)
        direction=-grad
        nombre=0
        while(True):
            if NL.norm(grad)<=epsilon or nombre>=nombreIteration:
                return val, pointCourant, nombre
            else:
                nombre=nombre+1
                gamma=optimize.brent(lambda gamma:objectiveGamma(gamma,X0=pointCourant,direction=direction,f=g))
                pointCourant=pointCourant+gamma*direction
                norm1=NL.norm(grad)**2
                val, grad=g(pointCourant)
                norm2=NL.norm(grad)**2
                b=norm2/norm1
                direction=-grad+b*direction

In [14]:
val,P_optimal,nombre=gradient_Conjuge(g=lambda P:movielensutils.objective(P,Q0=Q0,R=R,mask=mask,rho=rho),pointDepart=P0,epsilon=epsilon,nombreIteration=500)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 215963.131801    nombre d'itération: 9


### Question 3.3

In [15]:
import time
t0=time.clock()

print "Gradient with constant step size:"
t1=time.clock()
val,P_,nombre=gradient(g=lambda P:movielensutils.objective(P,Q0=Q0,R=R,mask=mask,rho=rho),P0=P0,gamma=gamma_donne,epsilon=epsilon,nombreIteration=500)
t2=time.clock()
duree=t2-t1
print("value_optimal:"+str(val))
print("nombreIteration:"+str(nombre))
print("run time:"+str(duree))

print("")
print "Gradient with line search:"
t1=time.clock()
val,P_optimal,nombre=gradient_LinearSearch(g=lambda P:movielensutils.objective(P,Q0=Q0,R=R,mask=mask,rho=rho),pointDepart=P0,epsilon=epsilon,nombreIteration=500)
t2=time.clock()
duree=t2-t1
print("value_optimal:"+str(val))
print("nombreIteration:"+str(nombre))
print("run time:"+str(duree))

print("")
print "Gradient conjugé:"
t1=time.clock()
val,P_optimal,nombre=gradient_Conjuge(g=lambda P:movielensutils.objective(P,Q0=Q0,R=R,mask=mask,rho=rho),pointDepart=P0,epsilon=epsilon,nombreIteration=500)
t2=time.clock()
duree=t2-t1
print("value_optimal:"+str(val))
print("nombreIteration:"+str(nombre))
print("run time:"+str(duree))



Gradient with constant step size:
value_optimal:215965.602538
nombreIteration:90
run time:3.31

Gradient with line search:
value_optimal:215963.767755
nombreIteration:17
run time:12.08

Gradient conjugé:
value_optimal:215963.131801
nombreIteration:9
run time:5.27


**Remarques:**
1. On trouve que tous les trois algorithmes peuvent atteindre la valeur optimale, mais avec le temps du calcul et le nombre d'itération différentes.
2. Il y a pas beaucoup de différences entre les valeurs optimales obtenues. Mais on peut trouver que "Line Searche" et "gradient conjugé" sont un peu mieux que "gradient with constant step".
3. Pour le temps du calcul, "gradient constant step" est plus rapide que "gradient conjugé", et "gradient conjugé" est plus rapide que "line search". Mais en fait, pour "gradient constant step", on doit passer pas mal de temps pour chercher le constant de lipschitzien. Donc, pour le temps du calcul total, "gradient constant step" ne vais pas donner un mieux résultat que les autres.
4. Pour la vitesse de convergence, "gradient conjugé" est mieux que "line search", et les deux premiers sont beaucoup mieux que "gradient constant step".

## 4 Résolution du problème complet

### Question 4.1

In [16]:
PQ0=np.concatenate([P0.ravel(), Q0.ravel()])
val, PQ, nombre = gradient_LinearSearch(lambda PQvec:total_objective_vectorized(PQvec, R=R, mask=mask, rho=rho),pointDepart=PQ0,epsilon=epsilon,nombreIteration=500)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 25633.2541509    nombre d'itération: 500


In [17]:
PQ0=np.concatenate([P0.ravel(), Q0.ravel()])
val, PQ, nombre = gradient_LinearSearch(lambda PQvec:total_objective_vectorized(PQvec, R=R, mask=mask, rho=rho),pointDepart=PQ0,epsilon=epsilon,nombreIteration=1000)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 24843.9936788    nombre d'itération: 1000


In [18]:
PQ0=np.concatenate([P0.ravel(), Q0.ravel()])
val, PQ, nombre = gradient_LinearSearch(lambda PQvec:total_objective_vectorized(PQvec, R=R, mask=mask, rho=rho),pointDepart=PQ0,epsilon=epsilon,nombreIteration=1500)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 24542.7814544    nombre d'itération: 1500


**Remarques:**
1. L'algorithme va renvoyer une valeur optimal, un vecteur PQ, et un nombre d'itération. Ici, le vecteur PQ est un vecteur de taille (9430+16820)=(26250), où les composants sont les valeurs de P et puis Q. Donc, pour avoir Q.dot(P), il faut reconstruire les matrices P et Q à partir ce vecteur PQ.
2. La vitesse de convergence n'est pas vite. Mais on peut sentir que la valeur optimale n'est pas loin que 24542.(pour nombre d'itération=5000, on a 24017).

### Question 4.2

**Reponses:**
1. On peut savoir que f(P1,Q0) <= f(P0,Q0), parce que quand Q=Q0, P1 minimise la valeur du f parmi les autres P. Même raisonnemnt, f(P1,Q1) <= f(P1,Q0), parce que Q1 minimise f quand P=P1. Ainsi, f(P2,Q1) <= f(P1,Q1)... Donc, par récurrence, on peut savoir que f décroit à chaque itération.
2. Comme f décroit à chaque itération et on a un valeur minimal du f, donc elle converge.

### Question 4.3

**REMARQUES:** 
1. Ici, j'utilise un critère d'un condition d'arrêt différent que les autres algorithmes. Je calcul la différrence des valeurs obtenus par deux successives appels de "gradient_LinearSearch à Q ou P fixé". Si la différence est inférieur à epsilon, on sort. (on peut aussi vérifier que les deux gradients obténus sont tous inférieurs à epsilon).
2. De plus, je compte le nombre d'itération total. Je mets à jour ce nombre après chaque appel de "gradient_LinearSearch": j'ajoute à ce nombre par le nombre d'itération utilisé dans cet appel.

In [19]:
def objective_Q(Q, P0, R, mask, rho):
    val, grad_P, grad_Q=total_objective(P=P0,Q=Q,R=R,mask=mask,rho=rho)
    return val, grad_Q



def gradient_MoindreCarre(P0,Q0,R,mask,rho,epsilon, nombreIteration=False):
    if nombreIteration==False :
        
        nombre=0
        val_avant=0
        P_courant=P0
        Q_courant=Q0
        
        val1,P_optimal,nbr=gradient_LinearSearch(g=lambda P:movielensutils.objective(P,Q0=Q_courant,R=R,mask=mask,rho=rho),pointDepart=P_courant,epsilon=epsilon,nombreIteration=500)
        nombre=nombre+nbr
        P_courant=P_optimal
        
        val2, Q_optimal,nbr= gradient_LinearSearch(g=lambda Q:objective_Q(Q,P0=P_courant,R=R,mask=mask,rho=rho),pointDepart=Q_courant,epsilon=epsilon,nombreIteration=500)
        nombre=nombre+nbr
        Q_courant=Q_optimal
        
        difference=np.abs(val1-val2)
        
        while(True):
            if difference<=epsilon :
                return val2,P_courant,Q_courant,nombre
            else:
                val1,P_optimal,nbr=gradient_LinearSearch(g=lambda P:movielensutils.objective(P,Q0=Q_courant,R=R,mask=mask,rho=rho),pointDepart=P_courant,epsilon=epsilon,nombreIteration=500)
                nombre=nombre+nbr
                P_courant=P_optimal

                val2, Q_optimal,nbr= gradient_LinearSearch(g=lambda Q:objective_Q(Q,P0=P_courant,R=R,mask=mask,rho=rho),pointDepart=Q_courant,epsilon=epsilon,nombreIteration=500)
                nombre=nombre+nbr
                Q_courant=Q_optimal

                difference=np.abs(val1-val2)
                

    else:          
        nombre=0
        val_avant=0
        P_courant=P0
        Q_courant=Q0
        
        val1,P_optimal,nbr=gradient_LinearSearch(g=lambda P:movielensutils.objective(P,Q0=Q_courant,R=R,mask=mask,rho=rho),pointDepart=P_courant,epsilon=epsilon,nombreIteration=500)
        nombre=nombre+nbr
        P_courant=P_optimal
        
        val2, Q_optimal,nbr= gradient_LinearSearch(g=lambda Q:objective_Q(Q,P0=P_courant,R=R,mask=mask,rho=rho),pointDepart=Q_courant,epsilon=epsilon,nombreIteration=500)
        nombre=nombre+nbr
        Q_courant=Q_optimal
        
        difference=np.abs(val1-val2)
        
        while(True):
            if difference<=epsilon or nombre >= nombreIteration:
                return val2,P_courant,Q_courant,nombre
            else:
                val1,P_optimal,nbr=gradient_LinearSearch(g=lambda P:movielensutils.objective(P,Q0=Q_courant,R=R,mask=mask,rho=rho),pointDepart=P_courant,epsilon=epsilon,nombreIteration=500)
                nombre=nombre+nbr
                P_courant=P_optimal

                val2, Q_optimal,nbr= gradient_LinearSearch(g=lambda Q:objective_Q(Q,P0=P_courant,R=R,mask=mask,rho=rho),pointDepart=Q_courant,epsilon=epsilon,nombreIteration=500)
                nombre=nombre+nbr
                Q_courant=Q_optimal

                difference=np.abs(val1-val2)

In [20]:
val, P, Q, nombre=gradient_MoindreCarre(P0=P0,Q0=Q0,R=R,mask=mask,rho=rho,epsilon=10., nombreIteration=500)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 129579.562278    nombre d'itération: 509


In [21]:
val, P, Q, nombre=gradient_MoindreCarre(P0=P0,Q0=Q0,R=R,mask=mask,rho=rho,epsilon=10., nombreIteration=1000)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 70642.6342025    nombre d'itération: 1097


In [22]:
val, P, Q, nombre=gradient_MoindreCarre(P0=P0,Q0=Q0,R=R,mask=mask,rho=rho,epsilon=10., nombreIteration=1500)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 53357.4072155    nombre d'itération: 1719


**Remarque:** Il converge un peu lentement en considérant le nombre d'itération.

### Question 4.4

In [23]:
val, P, Q, nombre=gradient_MoindreCarre(P0=P0,Q0=Q0,R=R,mask=mask,rho=rho,epsilon=10., nombreIteration=5000)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 37494.6342925    nombre d'itération: 5505


In [24]:
PQ0=np.concatenate([P0.ravel(), Q0.ravel()])
val, PQ, nombre = gradient_LinearSearch(lambda PQvec:total_objective_vectorized(PQvec, R=R, mask=mask, rho=rho),pointDepart=PQ0,epsilon=epsilon,nombreIteration=5000)
print("valeur optimale: "+str(val)+"    nombre d'itération: "+str(nombre))

valeur optimale: 24017.3159679    nombre d'itération: 5000


**Remarques:** On peut voir que la méthode de moindre carré alterné converge plus lentement que gradient avec recherche linéaire. 

### Question 4.5

**Remarques:** Comme le sujet a déjà fourni la valeur du rho, je suppose que l'on n'a pas besoin de varier cette valeur pour trouver une meilleur valeur optimale du fonction. C'est à dire, on n'a pas besoin de faire (juste pour montrer ce que on doit faire si le sujet ne donne pas la valeur du rho):
1. Séparer les données en deux parties, un pour apprentissage, un pour validation(chercher rho).
2. Pour un rho choisi, faire apprentissage et puis le test par norm(R-QP) en utilisant les données de validation. On va obtenir une valeur d'erreur( norm(R-QP) en les données de validation ).
3. On fait 3 avec des rhos différents et choisit un rho qui a une valeur d'erreur minimale.
4. On utilise ce rho pour prédire, c'est à dire prédire les notes des films qui ne sont pas données par un utilisateur choisi.

In [25]:
n_items = R.shape[1]
n_users = R.shape[0]
F = PQ.shape[0] / (n_items + n_users)
Pvec = PQ[0:n_items*F]
Qvec = PQ[n_items*F:]
P = np.reshape(Pvec, (F, n_items))
Q = np.reshape(Qvec, (n_users, F))

result=Q.dot(P)

In [26]:
print result[449]

[ 4.25507499  3.5448692   3.42647779 ...,  0.36835218  2.50951649
  2.7764629 ]


**On cherche les fillms qui ont notes supérieurs à 4.5 et qui n'avait pas un note avant, pour utilisateur 449.**

In [27]:
films_recommande=[]
notess=[]
for i in np.arange(n_items):
    if R[449][i]==0 and result[449][i]>=4.5:
        films_recommande.append(i)
        notess.append(result[449][i])

print films_recommande
print notess

[235, 245, 250, 255, 336, 407, 463, 514, 530, 675, 732, 908, 954, 957, 962, 1004, 1005, 1063, 1130, 1133, 1165, 1166, 1168, 1175, 1188, 1193, 1194, 1216, 1232, 1250, 1277, 1280, 1393, 1397, 1425, 1430, 1448, 1450, 1462, 1473, 1515, 1530, 1588, 1593, 1641, 1642]
[4.5605072765180674, 4.6225599970884366, 4.5250448757269703, 4.6308181407912805, 4.7459618283566307, 4.82981727158817, 5.1520874643615571, 4.6171470144388485, 4.5105559246866935, 4.8552461834053675, 4.5398405330586131, 4.5164215438324762, 5.2338648995518664, 4.6659272667826475, 5.1847780908789032, 5.1760353245053476, 4.7030326223363863, 4.6823211656564521, 4.5740756996151157, 4.7046132080138303, 4.8680789952428469, 4.7881642845548651, 4.71852110755857, 4.7288868633186674, 4.9010945205290852, 4.7241627231127366, 4.5286585668569828, 4.6926540183137693, 4.9154498616382689, 5.1683873009780452, 4.6445100231548357, 4.502034452088683, 4.6447272803557826, 4.5298912493782915, 4.549042073716814, 4.7089980292570548, 4.8369742907468352, 4.6

**On affiche le film qui a le note le plus grand.**

In [29]:
max_value = max(notess)
max_index = notess.index(max_value)
film_best=films_recommande[max_index]
print("film à recommender: "+str(film_best))
print("avec notes: "+str(max_value))

film à recommender: 1642
avec notes: 5.43257893273
